# Sparse Retrieval using Elastic Search

## Goals

- Understand the Python Elastic Search Client
- Map BM25 to Elastic Search 
- Compute Evaluation metrics 
- Other users of Elastic Search

## Imports

In [1]:
import pandas as pd
import tqdm.auto
import numpy as np
import glob
import concurrent.futures
import multiprocessing
import requests
from elasticsearch import Elasticsearch
from elasticsearch.helpers import streaming_bulk
import pprint

In [2]:
pd.options.display.max_colwidth = 500 # increase column width

In [3]:
from pandarallel import pandarallel

pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


## Data

For this workshop, we have two file.s

`posts.parquet` : contains a subset of stackoverflow posts

`related_posts.parquet`: contains questions pairs that were marked as duplicates

In [4]:
path_posts = "gs://np-training-tmp/stackoverflow/final_subset/posts.parquet"
path_posts_related = "gs://np-training-tmp/stackoverflow/final_subset/related_posts.parquet"

In [5]:
#ELASTIC_HOST="np-database.c.np-training.internal"
ELASTIC_HOST="localhost"
ELASTIC_INDEX="stackoverflow"
ELASTIC_PORT=9200

In [6]:
def create_index(client,index:str, num_shards=3):
    """Creates an index in Elasticsearch. Delete old index."""
    
    client.indices.delete(index=index)
    
    client.indices.create(
        index=index
        ,settings = {"number_of_shards": num_shards}
            # "mappings": {
            #     "properties": {
            #         "name": {"type": "text"},
            #         "borough": {"type": "keyword"},
            #         "cuisine": {"type": "keyword"},
            #         "grade": {"type": "keyword"},
            #         "location": {"type": "geo_point"},
            #     }
            # },
       
        #,ignore=400
    )


def generate_docs(df:pd.DataFrame):
    """
    Given a datframe containing posts data, yields a generator of dicitionary 
    """
    
    # iterate over dataframe contains posts with metadata
    for index, row in df.iterrows():
        doc = {**row} 
        
        # use PostId as document id
        doc['_id'] = doc["Id"]
        
        for k in list(doc.keys()):
            # don't insert nan fields
            if type(doc[k]) !=list and (doc[k] ==None or  ( pd.isna( doc[k] )  )) :
                del doc[k]
                
        del doc['Id']
        yield doc
        


def fetch_results(client:Elasticsearch, query:str,  num_hits=5, fields = ["Title", "QuestionBody"]):
    """
    With the passed elastic search client, return documents that contain the passed `query` in the fields specified by `fields`

    If the fields is empty, it will search all text fields
    """



    
    resp = client.search(
        query= {
                "multi_match": {
                    "query": query,
                    "fields": fields,
                }
            }
        ,size = num_hits
    )
    
    return resp
    

        

## Index Documents

In [7]:
df = pd.read_parquet(path_posts)
df['Tags']  = df['Tags'].apply(lambda x: x.tolist())

In [8]:
df.head()

,Id,AcceptedAnswerId,Title,QuestionBody,Tags,ViewCount,AnswerCount,CommentCount,Score,CreationDate,AnswerId,AcceptedAnswerBody
1,15020895,NaN,Python int-byte efficient data structure,"i am currently storing key-values of type int-int. For fast access, I am currently using an BTrees.IIBTree structure stored in memory. It is not stored on disk at all since we need the most recent data.\n\nHowever, the current solution barely fits into memory, so I am looking for a more efficient database or data structure in terms of access time. In case it would be stored in memory it also needs to be efficient in terms of memory space. \n\nOne idea would be to replace the BTrees.IIBTree s...","[python, data-structures]",155,0,3,1,2013-02-22 09:33:26.360,NaN,None
9,68487902,NaN,Why does the Variance of Laplace very different for OpenCV and scikit-image?,"TL;DR: How can I use skimage.filters.laplace(image).var() in a way to get the same value as cv2.Laplacian(image, CV_64F).var() and skimage.filters.sobel(image) to get same value as cv2.Sobel(image) ?\nI have the following code to find the Laplace Variance for blur detection\n[CODE]\nSo when I try to find the Laplace variance from OpenCV and scikit-image, it gives me two different values:\n[CODE]\nWhich one should I use or how can I get same number from both the functions?\nAlso, How can I us...","[python, opencv, image-processing, computer-vision, scikit-image]",391,0,5,1,2021-07-22 15:50:34.220,NaN,None
15,61391327,NaN,Why input never ends,"I have python 3.7 installed and I have this code:\n\n[CODE]\n\nI was writing the name and press enter but the input is not over, it is still running and waiting for more inputs\n\nEdit: the problem is that input is never ending, doesn't matter how many enters I press\n","[python, python-3.x, input]",104,1,6,3,2020-04-23 15:43:03.497,NaN,None
27,28852710,NaN,Crashes with piecewise linear objective for gurobi 6.0.2 / setPWLObj,"We have a complex optimization problem which includes several quadratic terms with integer and continous variables (using Anaconda Python / Pycharm with Gurobi 6.0.2). We applied the setPWLObj function to apprixmate the quadratic objective components. The code for this is as follows:\n\n[CODE]\n\nWith l1 and l2 being continous variables.\n\nThe problem behaves inconsistently. Running it on a Mac mostly delivers the exit codes 138 and 139 (correspondent to Bus Error 10), sometimes the same pr...","[python, crash, gurobi, piecewise]",403,1,1,3,2015-03-04 10:58:16.370,NaN,None
29,24043029,NaN,Python TypeError: plotdatehist() got an unexpected keyword argument,"apologies beforehand if this is a stupid question...\n\nI've been using some Manchester University code to record, analyse, and graphically display bird box activity using IR emitters/receivers using a Raspberry Pi.\nAnyway, I've run into a problem in the graphical display part. \n\nThe part of the code causing the error is: \n\n[CODE]\n\nand the error which keeps coming up reads\n\n[CODE]\n\nI've heard that similar problems can be fixed by updating software, but as far as I can tell everyth...","[python, typeerror]",419,0,7,0,2014-06-04 16:42:32.257,NaN,None


A sample document from our input file.  

The main fields we will searching against are the `Title` and `QuestionBody`

In [9]:
df.iloc[0].to_dict()

{'Id': 15020895,
 'AcceptedAnswerId': nan,
 'Title': 'Python int-byte efficient data structure',
 'QuestionBody': 'i am currently storing key-values of type int-int. For fast access, I am currently using an BTrees.IIBTree structure stored in memory. It is not stored on disk at all since we need the most recent data.\n\nHowever, the current solution barely fits into memory, so I am looking for a more efficient database or data structure in terms of access time. In case it would be stored in memory it also needs to be efficient in terms of memory space. \n\nOne idea would be to replace the BTrees.IIBTree structure with a int-byte hash written in C as an extension for Python, but the data would still be lost in case the machine fails (not a terrible thing in our case).\n\nWhat are your suggestions?\n',
 'Tags': ['python', 'data-structures'],
 'ViewCount': 155,
 'AnswerCount': 0,
 'CommentCount': 3,
 'Score': 1,
 'CreationDate': Timestamp('2013-02-22 09:33:26.360000'),
 'AnswerId': nan,
 '

In [10]:
?Elasticsearch

Init signature:
Elasticsearch(
    hosts: Union[str, List[Union[str, Mapping[str, Union[str, int]], elastic_transport.NodeConfig]], NoneType] = None,
    *,
    cloud_id: Union[str, NoneType] = None,
    api_key: Union[str, Tuple[str, str], NoneType] = None,
    basic_auth: Union[str, Tuple[str, str], NoneType] = None,
    bearer_auth: Union[str, NoneType] = None,
    opaque_id: Union[str, NoneType] = None,
    headers: Union[elastic_transport.client_utils.DefaultType, Mapping[str, str]] = <DEFAULT>,
    connections_per_node: Union[elastic_transport.client_utils.DefaultType, int] = <DEFAULT>,
    http_compress: Union[elastic_transport.client_utils.DefaultType, bool] = <DEFAULT>,
    verify_certs: Union[elastic_transport.client_utils.DefaultType, bool] = <DEFAULT>,
    ca_certs: Union[elastic_transport.client_utils.DefaultType, str] = <DEFAULT>,
    client_cert: Union[elastic_transport.client_utils.DefaultType, str] = <DEFAULT>,
    client_key: Union[elastic_transport.client_utils.Defau

create a client object to our elastic search cluster

In [11]:
client = Elasticsearch(
    [f'http://{ELASTIC_HOST}:{ELASTIC_PORT}']
)

tell elastic search to create an index     
An ES index is a collection of documents. 

ES suports inferring the documents without specifying the schema before hand 

In [12]:
create_index(client, index= ELASTIC_INDEX)

In [13]:
?client.indices.create

Signature:
client.indices.create(
    *,
    index: str,
    aliases: Union[Mapping[str, Mapping[str, Any]], NoneType] = None,
    error_trace: Union[bool, NoneType] = None,
    filter_path: Union[str, List[str], Tuple[str, ...], NoneType] = None,
    human: Union[bool, NoneType] = None,
    mappings: Union[Mapping[str, Any], NoneType] = None,
    master_timeout: Union[ForwardRef('t.Literal[-1]'), ForwardRef('t.Literal[0]'), str, NoneType] = None,
    pretty: Union[bool, NoneType] = None,
    settings: Union[Mapping[str, Any], NoneType] = None,
    timeout: Union[ForwardRef('t.Literal[-1]'), ForwardRef('t.Literal[0]'), str, NoneType] = None,
    wait_for_active_shards: Union[int, ForwardRef("t.Literal['all', 'index-setting']"), str, NoneType] = None,
) -> elastic_transport.ObjectApiResponse[typing.Any]
Docstring:
Creates an index with optional settings and mappings.

`<https://www.elastic.co/guide/en/elasticsearch/reference/8.4/indices-create-index.html>`_

:param index: The name of th

In [14]:
requests.get(f"http://{ELASTIC_HOST}:{ELASTIC_PORT}/_all/_settings").json()

{'test-index': {'settings': {'index': {'routing': {'allocation': {'include': {'_tier_preference': 'data_content'}}},
    'number_of_shards': '5',
    'provided_name': 'test-index',
    'creation_date': '1666564416488',
    'number_of_replicas': '1',
    'uuid': 'x7OXazwcTMWVVb7EeDW9pQ',
    'version': {'created': '8040399'}}}},
 'stackoverflow': {'settings': {'index': {'routing': {'allocation': {'include': {'_tier_preference': 'data_content'}}},
    'number_of_shards': '3',
    'provided_name': 'stackoverflow',
    'creation_date': '1667080909014',
    'number_of_replicas': '1',
    'uuid': 'TF1cw8NuQ36XHV9xCa7soA',
    'version': {'created': '8040399'}}}}}

The index we created is composed of `3` shards and `1` replica.   

When searching , ES queries each shard independantly and combines it

In [15]:
len(df)

219841

In [16]:
df_subset = df.head(5_000_000)
number_of_docs = len(df_subset)

Bulk insert all of our documents

In [17]:
with tqdm.auto.tqdm(total=number_of_docs , unit="docs" ) as pbar:
    successes = 0


    for ok, action in streaming_bulk(
            client=client, index=ELASTIC_INDEX, actions=generate_docs(df_subset) ,
        ):
        pbar.update(1)
        successes += ok


  0%|          | 0/219841 [00:00<?, ?docs/s]

Inserting `N` documents at `x` docs/seconf on a single node is pretty good

# Evaluate

In [18]:
?client.get

Signature:
client.get(
    *,
    index: str,
    id: str,
    error_trace: Union[bool, NoneType] = None,
    filter_path: Union[str, List[str], Tuple[str, ...], NoneType] = None,
    human: Union[bool, NoneType] = None,
    preference: Union[str, NoneType] = None,
    pretty: Union[bool, NoneType] = None,
    realtime: Union[bool, NoneType] = None,
    refresh: Union[bool, NoneType] = None,
    routing: Union[str, NoneType] = None,
    source: Union[bool, str, List[str], Tuple[str, ...], NoneType] = None,
    source_excludes: Union[str, List[str], Tuple[str, ...], NoneType] = None,
    source_includes: Union[str, List[str], Tuple[str, ...], NoneType] = None,
    stored_fields: Union[str, List[str], Tuple[str, ...], NoneType] = None,
    version: Union[int, NoneType] = None,
    version_type: Union[ForwardRef("t.Literal['external', 'external_gte', 'force', 'internal']"), str, NoneType] = None,
) -> elastic_transport.ObjectApiResponse[typing.Any]
Docstring:
Returns a document.

`<https:

getting a specific document by their id

In [19]:
resp = client.get(index=ELASTIC_INDEX, id=15020895)
resp.body

{'_index': 'stackoverflow',
 '_id': '15020895',
 '_version': 1,
 '_seq_no': 0,
 '_primary_term': 1,
 '_ignored': ['QuestionBody.keyword'],
 'found': True,
 '_source': {'Title': 'Python int-byte efficient data structure',
  'QuestionBody': 'i am currently storing key-values of type int-int. For fast access, I am currently using an BTrees.IIBTree structure stored in memory. It is not stored on disk at all since we need the most recent data.\n\nHowever, the current solution barely fits into memory, so I am looking for a more efficient database or data structure in terms of access time. In case it would be stored in memory it also needs to be efficient in terms of memory space. \n\nOne idea would be to replace the BTrees.IIBTree structure with a int-byte hash written in C as an extension for Python, but the data would still be lost in case the machine fails (not a terrible thing in our case).\n\nWhat are your suggestions?\n',
  'Tags': ['python', 'data-structures'],
  'ViewCount': 155,
  '

retrieve a document with a query

In [20]:
# https://stackoverflow.com/questions/34147471/elasticsearch-how-to-search-for-a-value-in-any-field-across-all-types-in-one


resp = client.search(
    query = {
            "multi_match": {
                "query": "pandas memmory issue",
                # "fields": ["Title", "QuestionBody"],
                            }
            }
    , size=5
    , explain=False
)

In [21]:
resp.body

{'took': 3569,
 'timed_out': False,
 '_shards': {'total': 8, 'successful': 8, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 10000, 'relation': 'gte'},
  'max_score': 13.040086,
  'hits': [{'_index': 'stackoverflow',
    '_id': '50681803',
    '_score': 13.040086,
    '_ignored': ['QuestionBody.keyword'],
    '_source': {'Title': 'convert sql code to python pandas dataframe operation',
     'QuestionBody': 'So I am having trouble with processing and manipulating large ammount of data.\nMy table 1 consist of 2 milions records for example: \n\n[CODE]\n\nand another table with data: \n\n[CODE]\n\nI am creating a join:\n\n[CODE]\n\nThe result is a really big table (SELECT should return around 40milions rows\nThen I use groupby and filter method to further filter my records. I have a problem because I get MemmoryError when running my code. I was thinking about changing csv to better accomodate pandas dataframe (to avoid using sql) \nMy code example is below: \n\n[CODE]\n\nThe bigg

In [22]:
pdf_related = pd.read_parquet(path_posts_related)

In [23]:
pdf_related.head()

,PostId,PostTitle,RelatedPostIds,RelatedPostTitles,num_candidates
1,3494593,Shading a kernel density plot between two points.,"[3494593, 14863744, 14094644, 16504452, 48853178, 36948624, 47308146, 34029811, 31215748, 29499914, 41484896, 7787114, 27189453, 23680729, 36224394, 18742693]","[Shading a kernel density plot between two points., adding percentile lines to a density plot, draw the following shaded area in R, color a portion of the normal distribution, How can I shade the area under a curve?, Shade area under a curve, Shading a region under a PDF, Fill different colors for each quantile in geom_density() of ggplot, How to shade part of a density curve in ggplot (with no y axis data), r density plot - fill area under curve, Fill negative value area below geom_line, po...",16
2,37949409,Dictionary in a numpy array?,"[37949409, 47689224, 61517741]","[Dictionary in a numpy array?, How to access the elements in numpy array of sets?, opening npy array. can view but not index?]",3
8,19876079,Cannot find module cv2 when using OpenCV,"[19876079, 62443365, 64580641, 45606137, 60294113, 65227902, 63039959]","[Cannot find module cv2 when using OpenCV, How to use opencv module in python(I'm using pycharm), build opencv from source: ModuleNotFoundError: No module named 'cv2', ImportError: No module named cv2 when executing Python script, 'opencv-python' installed but still shows 'ModuleNotFoundError: No module named cv2 ', Installed OpenCV successfully, but cannot import it within modules, On raspberry pi terminal cv2 works but on my project didnt work how can i fix this]",7
12,35082143,Error: package or namespace load failed for ‘car’,"[35082143, 65941744, 68515009, 56409535]","[Error: package or namespace load failed for ‘car’, Error: package or namespace load failed for ‘tidyverse’ there is no package called ‘reprex’, Truble loading 'Hmisc', > library(ez) Error: package or namespace load failed for ‘ez’ in loadNamespace]",4
14,2673651,inheritance from str or int,"[2673651, 48465797, 3120562, 15085917, 3238350, 4827303, 29751474, 50051365, 5693942, 59567148, 30045106, 37764447, 65568299, 24736813, 38873373]","[inheritance from str or int, Inherited class of int doesn't take additional arguments, Python, subclassing immutable types, Inheriting from immutable types, Subclassing int in Python, problem subclassing builtin type, Customizing immutable types in Python, Class inheritance not working while creating a Dimension custom class with int parent class in Python 3.6, Subclassing int and overriding the __init__ method - Python, How to inherit class complex in python?, Python how to extend `str` an...",15


In [24]:
len (pdf_related)

6114

In [25]:
resp['hits']['hits'][0];

In [26]:
def format_resp(resp, row):
    payload = []
    query = row['PostTitle']
    for hit in resp['hits']['hits']:
        doc_id = int(hit['_id'])
        
        r = {
             'query': query
             , 'query_id' : row['PostId']
             ,'doc_id' : doc_id
             , 'is_relevant' : doc_id in row['RelatedPostIds']
             ,'score' : hit['_score']
             ,'doc_title' : hit['_source']['Title']


        }
        payload.append(r)    
    return payload

def fetch_as_relevancy_eval(row,num_hits=10):
    client = Elasticsearch(
    [f'http://{ELASTIC_HOST}:{ELASTIC_PORT}']
    
    )
    
    
    query = row['PostTitle']
    resp = fetch_results(client, query, num_hits=num_hits)
    payload = format_resp(resp, row)
    
    return pd.DataFrame(payload)
    

def evaluate_relevancy_hits(df,num_hits=10):
    
    payload = []
    for index, row in df.iterrows():

        payload_query = fetch_as_relevancy_eval(row)
        
        payload.extend(payload_query.to_dict(orient='records') )

    
    #return pd.DataFrame.from_records(payload)
    return pd.DataFrame(payload)
    



def evaluate_relevancy_hits2(df,num_hits=20):
    
    
    res = df.parallel_apply(fetch_as_relevancy_eval,num_hits=num_hits, axis = 1)

    return res
    

In [27]:
fetch_as_relevancy_eval(pdf_related.iloc[0].to_dict() )

,query,query_id,doc_id,is_relevant,score,doc_title
0,Shading a kernel density plot between two points.,3494593,14094644,True,49.934696,draw the following shaded area in R
1,Shading a kernel density plot between two points.,3494593,7787114,True,45.700478,polygon in density plot?
2,Shading a kernel density plot between two points.,3494593,3494593,True,45.222992,Shading a kernel density plot between two points.
3,Shading a kernel density plot between two points.,3494593,27294822,False,39.258953,Shading a kernel density estimate between two points - with transparency
4,Shading a kernel density plot between two points.,3494593,27189453,True,34.868088,Shade (fill or color) area under density curve by quantile
5,Shading a kernel density plot between two points.,3494593,30900745,False,24.448137,"Plot 2D-kernel density from a dataframe: set number of grid positions, bandwith and lims"
6,Shading a kernel density plot between two points.,3494593,35381762,False,24.032297,Are SciPy Kernel Density values dependent upon the density per unit area or volume when using 2D or 3D poiint data?
7,Shading a kernel density plot between two points.,3494593,29244352,False,23.956926,Python: Overlap between two functions (PDF of kde and normal)
8,Shading a kernel density plot between two points.,3494593,55131398,False,23.938930,matplotlib scatter: the more overlapping points the bigger the marker
9,Shading a kernel density plot between two points.,3494593,47644966,False,23.525637,Change color of seaborn distribution line


In [28]:
evaluate_relevancy_hits(pdf_related.iloc[0:2])

,query,query_id,doc_id,is_relevant,score,doc_title
0,Shading a kernel density plot between two points.,3494593,14094644,True,49.934696,draw the following shaded area in R
1,Shading a kernel density plot between two points.,3494593,7787114,True,45.700478,polygon in density plot?
2,Shading a kernel density plot between two points.,3494593,3494593,True,45.222992,Shading a kernel density plot between two points.
3,Shading a kernel density plot between two points.,3494593,27294822,False,39.258953,Shading a kernel density estimate between two points - with transparency
4,Shading a kernel density plot between two points.,3494593,27189453,True,34.868088,Shade (fill or color) area under density curve by quantile
5,Shading a kernel density plot between two points.,3494593,30900745,False,24.448137,"Plot 2D-kernel density from a dataframe: set number of grid positions, bandwith and lims"
6,Shading a kernel density plot between two points.,3494593,35381762,False,24.032297,Are SciPy Kernel Density values dependent upon the density per unit area or volume when using 2D or 3D poiint data?
7,Shading a kernel density plot between two points.,3494593,29244352,False,23.956926,Python: Overlap between two functions (PDF of kde and normal)
8,Shading a kernel density plot between two points.,3494593,55131398,False,23.938930,matplotlib scatter: the more overlapping points the bigger the marker
9,Shading a kernel density plot between two points.,3494593,47644966,False,23.525637,Change color of seaborn distribution line


In [29]:
resp = evaluate_relevancy_hits2(pdf_related.head(1000000) )

In [30]:
df_res  = pd.concat(list(resp) ,ignore_index = True)

In [31]:
df_res

,query,query_id,doc_id,is_relevant,score,doc_title
0,Shading a kernel density plot between two points.,3494593,14094644,True,49.934696,draw the following shaded area in R
1,Shading a kernel density plot between two points.,3494593,7787114,True,45.700478,polygon in density plot?
2,Shading a kernel density plot between two points.,3494593,3494593,True,45.222992,Shading a kernel density plot between two points.
3,Shading a kernel density plot between two points.,3494593,27294822,False,39.258953,Shading a kernel density estimate between two points - with transparency
4,Shading a kernel density plot between two points.,3494593,27189453,True,34.868088,Shade (fill or color) area under density curve by quantile
...,...,...,...,...,...,...
122275,pandas time shift from utc to local,37814201,41592506,False,19.179344,Get system time w/timezone in Django bypassing default timezone
122276,pandas time shift from utc to local,37814201,23497205,False,19.111530,Time conversion using pytz isn't accurate
122277,pandas time shift from utc to local,37814201,55521202,False,19.069027,Uploading to Bigquery with correctly provided timezone in timestamp (prevent auto UTC)
122278,pandas time shift from utc to local,37814201,40294769,False,18.964666,Converting local time to UTC using pytz adds DST?


In [32]:
query_id = 30212447

In [33]:
pdf_related [ pdf_related['PostId'] == query_id ].iloc[0].to_dict()

{'PostId': 30212447,
 'PostTitle': 'How to add element in Python to the end of list using list.insert?',
 'RelatedPostIds': array([30212447, 70342396, 64223356, 54052453, 53932704]),
 'RelatedPostTitles': array(['How to add element in Python to the end of list using list.insert?',
        'Some confusion about swapping two elements in a list using a function',
        'while using "-1" as index number,the element is inserting at last 2nd position. how its happening?',
        'Insert an item to the last but one position in list',
        'Array: Insert with negative index'], dtype=object),
 'num_candidates': 5}

In [34]:
df_res[ df_res.query_id==query_id]

,query,query_id,doc_id,is_relevant,score,doc_title
19980,How to add element in Python to the end of list using list.insert?,30212447,30212447,True,37.783924,How to add element in Python to the end of list using list.insert?
19981,How to add element in Python to the end of list using list.insert?,30212447,21939652,False,23.490698,Insert at first position of a list in Python
19982,How to add element in Python to the end of list using list.insert?,30212447,47621511,False,19.651970,Do Python list comprehensions append at each iteration?
19983,How to add element in Python to the end of list using list.insert?,30212447,70946087,False,19.495499,Python how to add quote to one of the element in a list?
19984,How to add element in Python to the end of list using list.insert?,30212447,23143011,False,18.778458,"Python - regex, blank element at the end of the list?"
19985,How to add element in Python to the end of list using list.insert?,30212447,24612665,False,18.514690,Insert function use with nested list
19986,How to add element in Python to the end of list using list.insert?,30212447,25495944,False,18.439268,How to add two element into a list using list comprehension in python
19987,How to add element in Python to the end of list using list.insert?,30212447,47440037,False,18.421326,Extending list by adding element to special position
19988,How to add element in Python to the end of list using list.insert?,30212447,63873586,False,17.913140,Returning smallest positive int that does not occur in given list
19989,How to add element in Python to the end of list using list.insert?,30212447,52957447,False,17.829090,Python print out integers with suffixes in the end


In [35]:
def metrics(result):
    
    result = list(result) 
    
    mrr = 0
    
    if True in result:
        first_index = result.index(True) 
        mrr = 1 /  (first_index + 1)
    
    res=  {
     "p@1" :  sum(result[:1])  
    , "p@5" :  sum(result[:5]) / 5 
    , "p@10" :  sum(result[:10])  / 10 
    , "mrr" :  mrr

        
        
    }
    return pd.Series(res)

In [36]:
#?df_res.groupby

In [37]:
df_agg_res  = df_res.groupby(['query_id'], as_index=False).apply (lambda x: metrics(x['is_relevant']))



In [38]:
df_agg_res.drop(columns='query_id').agg(np.mean)

p@1     0.983808
p@5     0.266078
p@10    0.146385
mrr     0.990946
dtype: float64

Queries where we didn't do weell

In [39]:
df_agg_res [  (df_agg_res['p@1'] < 1) & (df_agg_res['p@5'] > 0.4) ]

,query_id,p@1,p@5,p@10,mrr
2083,16819956,0.0,0.6,0.6,0.500000
3540,31087111,0.0,0.6,0.3,0.333333
3889,34837707,0.0,0.6,0.3,0.500000
4230,38744285,0.0,0.8,0.8,0.500000
4236,38824634,0.0,0.8,0.4,0.500000
4401,40809503,0.0,0.6,0.3,0.500000


In [40]:
query_id = 16819956

In [41]:
df_res [ df_res.query_id==query_id ].iloc[0]['query']

'Warning message: In `...` : invalid factor level, NA generated'

In [42]:
df_res [ df_res.query_id==query_id ][['doc_title','is_relevant'] ]

,doc_title,is_relevant
18360,using rbind gives error warning about factor levels,False
18361,"Warning message: In `...` : invalid factor level, NA generated",True
18362,"R warning message - invalid factor level, NA generated",False
18363,"Invalid factor level, NA generated warning",True
18364,grepl in R: Replace character/numeric levels,True
18365,Setting categories from another column values,False
18366,R factor levels,True
18367,How do I change NA to 0 in R,True
18368,"invalid factor level, NA generated for only one level in a factor variable",True
18369,"""invalid factor level, NA generated"" error when trying to link colour information in with subgroup",False


In [43]:
df_res[ df_res.query_id==query_id]

,query,query_id,doc_id,is_relevant,score,doc_title
18360,"Warning message: In `...` : invalid factor level, NA generated",16819956,55746092,False,47.580658,using rbind gives error warning about factor levels
18361,"Warning message: In `...` : invalid factor level, NA generated",16819956,16819956,True,45.562363,"Warning message: In `...` : invalid factor level, NA generated"
18362,"Warning message: In `...` : invalid factor level, NA generated",16819956,29569624,False,44.619026,"R warning message - invalid factor level, NA generated"
18363,"Warning message: In `...` : invalid factor level, NA generated",16819956,25729700,True,42.956326,"Invalid factor level, NA generated warning"
18364,"Warning message: In `...` : invalid factor level, NA generated",16819956,57675005,True,42.222477,grepl in R: Replace character/numeric levels
18365,"Warning message: In `...` : invalid factor level, NA generated",16819956,60008033,False,36.226370,Setting categories from another column values
18366,"Warning message: In `...` : invalid factor level, NA generated",16819956,33965149,True,34.363070,R factor levels
18367,"Warning message: In `...` : invalid factor level, NA generated",16819956,58943375,True,34.220978,How do I change NA to 0 in R
18368,"Warning message: In `...` : invalid factor level, NA generated",16819956,65610451,True,33.731857,"invalid factor level, NA generated for only one level in a factor variable"
18369,"Warning message: In `...` : invalid factor level, NA generated",16819956,59157536,False,33.572630,"""invalid factor level, NA generated"" error when trying to link colour information in with subgroup"


In [44]:
df.to_parquet("../tmp/posts.parquet", index=False)
df.head()

,Id,AcceptedAnswerId,Title,QuestionBody,Tags,ViewCount,AnswerCount,CommentCount,Score,CreationDate,AnswerId,AcceptedAnswerBody
1,15020895,NaN,Python int-byte efficient data structure,"i am currently storing key-values of type int-int. For fast access, I am currently using an BTrees.IIBTree structure stored in memory. It is not stored on disk at all since we need the most recent data.\n\nHowever, the current solution barely fits into memory, so I am looking for a more efficient database or data structure in terms of access time. In case it would be stored in memory it also needs to be efficient in terms of memory space. \n\nOne idea would be to replace the BTrees.IIBTree s...","[python, data-structures]",155,0,3,1,2013-02-22 09:33:26.360,NaN,None
9,68487902,NaN,Why does the Variance of Laplace very different for OpenCV and scikit-image?,"TL;DR: How can I use skimage.filters.laplace(image).var() in a way to get the same value as cv2.Laplacian(image, CV_64F).var() and skimage.filters.sobel(image) to get same value as cv2.Sobel(image) ?\nI have the following code to find the Laplace Variance for blur detection\n[CODE]\nSo when I try to find the Laplace variance from OpenCV and scikit-image, it gives me two different values:\n[CODE]\nWhich one should I use or how can I get same number from both the functions?\nAlso, How can I us...","[python, opencv, image-processing, computer-vision, scikit-image]",391,0,5,1,2021-07-22 15:50:34.220,NaN,None
15,61391327,NaN,Why input never ends,"I have python 3.7 installed and I have this code:\n\n[CODE]\n\nI was writing the name and press enter but the input is not over, it is still running and waiting for more inputs\n\nEdit: the problem is that input is never ending, doesn't matter how many enters I press\n","[python, python-3.x, input]",104,1,6,3,2020-04-23 15:43:03.497,NaN,None
27,28852710,NaN,Crashes with piecewise linear objective for gurobi 6.0.2 / setPWLObj,"We have a complex optimization problem which includes several quadratic terms with integer and continous variables (using Anaconda Python / Pycharm with Gurobi 6.0.2). We applied the setPWLObj function to apprixmate the quadratic objective components. The code for this is as follows:\n\n[CODE]\n\nWith l1 and l2 being continous variables.\n\nThe problem behaves inconsistently. Running it on a Mac mostly delivers the exit codes 138 and 139 (correspondent to Bus Error 10), sometimes the same pr...","[python, crash, gurobi, piecewise]",403,1,1,3,2015-03-04 10:58:16.370,NaN,None
29,24043029,NaN,Python TypeError: plotdatehist() got an unexpected keyword argument,"apologies beforehand if this is a stupid question...\n\nI've been using some Manchester University code to record, analyse, and graphically display bird box activity using IR emitters/receivers using a Raspberry Pi.\nAnyway, I've run into a problem in the graphical display part. \n\nThe part of the code causing the error is: \n\n[CODE]\n\nand the error which keeps coming up reads\n\n[CODE]\n\nI've heard that similar problems can be fixed by updating software, but as far as I can tell everyth...","[python, typeerror]",419,0,7,0,2014-06-04 16:42:32.257,NaN,None


In [45]:
pdf_related.to_parquet("../tmp/related.parquet", index=False)
pdf_related.head()

,PostId,PostTitle,RelatedPostIds,RelatedPostTitles,num_candidates
1,3494593,Shading a kernel density plot between two points.,"[3494593, 14863744, 14094644, 16504452, 48853178, 36948624, 47308146, 34029811, 31215748, 29499914, 41484896, 7787114, 27189453, 23680729, 36224394, 18742693]","[Shading a kernel density plot between two points., adding percentile lines to a density plot, draw the following shaded area in R, color a portion of the normal distribution, How can I shade the area under a curve?, Shade area under a curve, Shading a region under a PDF, Fill different colors for each quantile in geom_density() of ggplot, How to shade part of a density curve in ggplot (with no y axis data), r density plot - fill area under curve, Fill negative value area below geom_line, po...",16
2,37949409,Dictionary in a numpy array?,"[37949409, 47689224, 61517741]","[Dictionary in a numpy array?, How to access the elements in numpy array of sets?, opening npy array. can view but not index?]",3
8,19876079,Cannot find module cv2 when using OpenCV,"[19876079, 62443365, 64580641, 45606137, 60294113, 65227902, 63039959]","[Cannot find module cv2 when using OpenCV, How to use opencv module in python(I'm using pycharm), build opencv from source: ModuleNotFoundError: No module named 'cv2', ImportError: No module named cv2 when executing Python script, 'opencv-python' installed but still shows 'ModuleNotFoundError: No module named cv2 ', Installed OpenCV successfully, but cannot import it within modules, On raspberry pi terminal cv2 works but on my project didnt work how can i fix this]",7
12,35082143,Error: package or namespace load failed for ‘car’,"[35082143, 65941744, 68515009, 56409535]","[Error: package or namespace load failed for ‘car’, Error: package or namespace load failed for ‘tidyverse’ there is no package called ‘reprex’, Truble loading 'Hmisc', > library(ez) Error: package or namespace load failed for ‘ez’ in loadNamespace]",4
14,2673651,inheritance from str or int,"[2673651, 48465797, 3120562, 15085917, 3238350, 4827303, 29751474, 50051365, 5693942, 59567148, 30045106, 37764447, 65568299, 24736813, 38873373]","[inheritance from str or int, Inherited class of int doesn't take additional arguments, Python, subclassing immutable types, Inheriting from immutable types, Subclassing int in Python, problem subclassing builtin type, Customizing immutable types in Python, Class inheritance not working while creating a Dimension custom class with int parent class in Python 3.6, Subclassing int and overriding the __init__ method - Python, How to inherit class complex in python?, Python how to extend `str` an...",15


In [46]:
df_agg_res.to_parquet("../tmp/df_agg_res__elasticsearch.parquet", index=False)
df_agg_res.head()

,query_id,p@1,p@5,p@10,mrr
0,972,1.0,0.2,0.3,1.0
1,8948,1.0,0.4,0.3,1.0
2,20794,1.0,0.2,0.1,1.0
3,32404,1.0,0.4,0.3,1.0
4,32899,1.0,0.8,0.4,1.0


In [47]:
df_res.to_parquet("../tmp/df_res__elasticsearch.parquet", index=False)
df_res.head()

,query,query_id,doc_id,is_relevant,score,doc_title
0,Shading a kernel density plot between two points.,3494593,14094644,True,49.934696,draw the following shaded area in R
1,Shading a kernel density plot between two points.,3494593,7787114,True,45.700478,polygon in density plot?
2,Shading a kernel density plot between two points.,3494593,3494593,True,45.222992,Shading a kernel density plot between two points.
3,Shading a kernel density plot between two points.,3494593,27294822,False,39.258953,Shading a kernel density estimate between two points - with transparency
4,Shading a kernel density plot between two points.,3494593,27189453,True,34.868088,Shade (fill or color) area under density curve by quantile


## Conclusion

Hope this notebook , showed how simple it is to implement a Sparse Retriever using Elastic Search